In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/contraction"
results_dir = "results_contraction"
model_names = ["Dirichlet Horseshoe", "Dirichlet Horseshoe full"]

all_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "config_1/GAM_N100_p8_sigma1.00_seed1"
    print(full_config_path)
    print(base_config_name)
    fits = get_model_fits(
        config=full_config_path,
        results_dir=results_dir,
        models=model_names,
        include_prior=False,
    )

    all_fits[base_config_name] = fits  # use clean key



In [3]:
def get_N_sigma(seed):
    if seed == 1:
        N=25
    elif seed == 2:
        N=50
    elif seed == 3:
        N=100
    elif seed == 4:
        N=200
    sigma=1.0
    return N, sigma

def compute_rmse_results(seeds, models, all_fits, get_N_sigma):
    results = []
    posterior_means = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed)
        dataset_key = f'INT_N{N}_p8_sigma{sigma:.2f}_seed{seed}'
        path = f"datasets/contraction/{dataset_key}.npz"

        try:
            data = np.load(path)
            y_test = data["y_test"]
        except FileNotFoundError:
            print(f"[SKIP] File not found: {path}")
            continue

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                output_test = fit.stan_variable("output_test")  # (S, N/5, O)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = output_test.shape[0]
            rmses = np.zeros(S)

            for i in range(S):
                y_hat = output_test[i]
                rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test.squeeze()) ** 2))
                
            posterior_mean = np.mean(output_test, axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean.squeeze() - y_test.squeeze()) ** 2))

            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'posterior_mean_rmse': posterior_mean_rmse
            })

            for i in range(S):
                results.append({
                    'seed': seed,
                    'N': N,
                    'sigma': sigma,
                    'model': model,
                    'rmse': rmses[i]
                })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse


In [4]:
seeds = [1, 2, 3, 4]


df_rmse, df_posterior_rmse = compute_rmse_results(
    seeds, model_names, all_fits, get_N_sigma
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Define the order for plotting
model_order = ["Dirichlet Horseshoe", "Dirichlet Horseshoe full"]
N_order = [25, 50, 100, 200]  # If you only have 25, 50, 100, adjust this

# Filter to include only sigma = 1.0 (as before)
df_plot = df_rmse[df_rmse["sigma"] == 1.0].copy()

# Optional: Only keep N values you actually have data for
df_plot = df_plot[df_plot["N"].isin([25, 50, 100, 200])]  # Add 200 if available

# Create a composite x-axis grouping
df_plot["N_str"] = df_plot["N"].astype(str)  # Make sure it's string for seaborn

plt.figure(figsize=(12, 6))

sns.boxplot(
    data=df_plot,
    x="N_str",
    y="rmse",
    hue="model",
    hue_order=model_order,
    order=[str(n) for n in N_order if n in df_plot["N"].unique()],
)

plt.title("RMSE by Sample Size N and Model")
plt.xlabel("Sample Size N")
plt.ylabel("RMSE")
plt.ylim(0, 8)
plt.grid(True, axis='y')

plt.legend(title="Model")
plt.tight_layout()
plt.show()


## COMPARE

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# 1. RMSE Computation Function
# -----------------------------

def compute_rmse_results(seeds, N_vals, models, all_fits, sigma=1.0, M_func=None):
    results = []
    posterior_means = []

    for seed in seeds:
        N = N_vals[seed]
        dataset_key = f'INT_N{N}_p8_sigma{sigma:.2f}_seed{seed}'

        try:
            data = np.load(f"datasets/contraction/{dataset_key}.npz")
            y_test = data["y_test"]
        except FileNotFoundError:
            print(f"[SKIP] Missing data: {dataset_key}")
            continue

        M = M_func(N) if M_func else 1.0

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                output_test = fit.stan_variable("output_test")
            except KeyError:
                print(f"[SKIP] Missing model/posterior: {dataset_key} -> {model}")
                continue

            S = output_test.shape[0]
            for i in range(S):
                rmse = np.sqrt(np.mean((output_test[i].squeeze() - y_test.squeeze())**2)) / M
                results.append({
                    'seed': seed, 'N': N, 'model': model, 'rmse': rmse
                })

            posterior_mean = output_test.mean(axis=0)
            pm_rmse = np.sqrt(np.mean((posterior_mean.squeeze() - y_test.squeeze())**2)) / M
            posterior_means.append({
                'seed': seed, 'N': N, 'model': model, 'posterior_mean_rmse': pm_rmse
            })

    df_rmse = pd.DataFrame(results)
    df_post = pd.DataFrame(posterior_means)

    return df_rmse, df_post

# -----------------------------
# 2. Plotting Function
# -----------------------------

def plot_contraction(df_post, models, alpha=1.0, p=8, M_func=None):
    fig, ax = plt.subplots(figsize=(8,6))

    Ns = sorted(df_post['N'].unique())

    for model in models:
        dfm = df_post[df_post['model'] == model]
        means = dfm.groupby('N')['posterior_mean_rmse'].mean()
        stds = dfm.groupby('N')['posterior_mean_rmse'].std()

        ax.errorbar(means.index, means, yerr=stds, label=model, marker='o', capsize=3)

    # Plot theoretical contraction curve
    slope = -alpha / (2 * alpha + p)
    ref_N = Ns[0]
    ref_val = df_post[df_post['N'] == ref_N]['posterior_mean_rmse'].mean()

    theory_x = np.array(Ns)
    theory_y = ref_val * (theory_x / ref_N) ** slope

    ax.plot(theory_x, theory_y, '--', color='gray', label=f'Theory: slope={slope:.2f}')

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Sample Size $N$ (log)')
    ax.set_ylabel('Scaled Posterior Mean RMSE (log)')
    ax.set_title('Empirical Posterior Contraction vs. Theoretical Rate')
    ax.legend()
    ax.grid(True, which='both', linestyle='--', alpha=0.5)

    return fig

# -----------------------------
# 3. Example Setup & Execution
# -----------------------------

# Example inputs (adjust these to your context):
N_vals = {1: 25, 2: 50, 3: 100, 4: 200, 5: 400}
seeds = [1, 2, 3, 4, 5]
models = ["Dirichlet Horseshoe", "Dirichlet Horseshoe full"]
sigma = 1.0
alpha = 1.0
p = 8

# Optional: define theoretical epsilon_n scaling
def epsilon_n(N):
    return N ** (-alpha / (2 * alpha + p))

# Assume you have thi


In [ ]:
# Compute results:
df_rmse, df_post = compute_rmse_results(
    seeds=seeds,
    N_vals=N_vals,
    models=models,
    all_fits=all_fits,
    sigma=sigma,
    M_func=epsilon_n  # Optional scaling
)

# Plot results:
fig = plot_contraction(df_post, models=models, alpha=alpha, p=p)
plt.show()

In [9]:
import pandas as pd
import numpy as np
import arviz as az

def get_N_sigma(seed):
    if seed == 1:
        N=25
    elif seed == 2:
        N=50
    elif seed == 3:
        N=100
    elif seed == 4:
        N=200
    sigma=1.0
    return N, sigma


def get_all_convergence_diagnostics(all_fits):
    diagnostics = []

    for config_name, model_fits in all_fits.items():
        for model_name, fit in model_fits.items():
            try:
                idata = az.from_cmdstanpy(fit['posterior'])
                y_pred = fit['posterior'].stan_variable('output_test')
                
                path = f'datasets/contraction/{config_name}.npz'
                try:
                    data = np.load(path)
                    y_test = data["y_test"]
                except FileNotFoundError:
                    print(f"[SKIP] File not found: {path}")
                    continue
                
                S = y_pred.shape[0]
                rmses = np.zeros(S)
                for i in range(S):
                   rmses[i] = np.sqrt(np.mean((y_pred[i].squeeze() - y_test.squeeze()) ** 2))

                summary = az.summary(idata, var_names=["output"], round_to=3)
                
                rhat = summary["r_hat"]

                ess_bulk = summary["ess_bulk"]
                ess_tail = summary["ess_tail"]
                
                try:
                    seed = int(config_name.split("_seed")[-1])
                    N, sigma = get_N_sigma(seed)
                except:
                    N, sigma = np.nan, np.nan

                diagnostics.append({
                    #"config": config_name,
                    "model": model_name,
                    "max_rhat": rhat.max(),
                    "median_rhat": rhat.median(),
                    #"p95_rhat": rhat.quantile(0.95),
                    "rmse": np.mean(rmses, axis=0),
                    #"min_ess_bulk": ess_bulk.min(),
                    "median_ess_bulk": ess_bulk.median(),
                    #"p05_ess_bulk": ess_bulk.quantile(0.05),
                    #"min_ess_tail": ess_tail.min(),
                    "median_ess_tail": ess_tail.median(),
                    "N": N,
                    "sigma": sigma,
                    #"p05_ess_tail": ess_tail.quantile(0.05),
                    #"n_divergent": divergences
                })

            except Exception as e:
                diagnostics.append({
                    #"config": config_name,
                    "model": model_name,
                    "max_rhat": np.nan,
                    "median_rhat": np.nan,
                    "p95_rhat": np.nan,
                    "min_ess_bulk": np.nan,
                    "min_ess_tail": np.nan,
                    #"n_divergent": np.nan,
                    "error": str(e)
                })

    return pd.DataFrame(diagnostics)


In [10]:
relu_diagostic = get_all_convergence_diagnostics(all_fits)

In [ ]:
relu_grouped = relu_diagostic.assign(row_index=lambda df: df.index) \
    .sort_values(["model", "N"]) \
    .drop(columns="row_index") \
    .reset_index(drop=True)
    
relu_grouped

In [ ]:
np.exp(-6*3)